In [1]:
from tools import get_data
data = get_data(3)

## Day 3: Crossed Wires
The gravity assist was successful, and you're well on your way to the Venus refuelling station. During the rush back on Earth, the fuel management system wasn't completely installed, so that's next on the priority list.

Opening the front panel reveals a jumble of wires. Specifically, two wires are connected to a central port and extend outward on a grid. You trace the path each wire takes as it leaves the central port, one wire per line of text (your puzzle input).

The wires twist and turn, but the two wires occasionally cross paths. To fix the circuit, you need to find the intersection point closest to the central port. Because the wires are on a grid, use the Manhattan distance for this measurement. While the wires do technically cross right at the central port where they both start, this point does not count, nor does a wire count as crossing with itself.

For example, if the first wire's path is R8,U5,L5,D3, then starting from the central port (o), it goes right 8, up 5, left 5, and finally down 3:

```
...........
...........
...........
....+----+.
....|....|.
....|....|.
....|....|.
.........|.
.o-------+.
...........
```

Then, if the second wire's path is U7,R6,D4,L4, it goes up 7, right 6, down 4, and left 4:

```
...........
.+-----+...
.|.....|...
.|..+--X-+.
.|..|..|.|.
.|.-X--+.|.
.|..|....|.
.|.......|.
.o-------+.
...........
```

These wires cross at two locations (marked X), but the lower-left one is closer to the central port: its distance is 3 + 3 = 6.

In [3]:
data.count('\n')

2

In [4]:
def manhattan_distance(p1, p2):
    x1, y1 = p1
    x2, y2 = p2
    return abs(x1 - x2) + abs(y1 - y2)

In [12]:
def all_wire_positions(path):
    current_location = (0, 0)
    all_positions = [] # we explicitly don't care about this one
    for motion in path:
        x, y = current_location
        direction, steps = motion
        if direction == 'R':
            positions = [(x+i+1, y) for i in range(steps)]
        elif direction == 'L':
            positions = [(x-i-1, y) for i in range(steps)]
        elif direction == 'U':
            positions = [(x, y+i+1) for i in range(steps)]
        elif direction == 'D':
            positions = [(x, y-i-1) for i in range(steps)]
        else:
            raise Exception(f"Unknown direction {direction}")
        current_location = positions[-1]
        all_positions.extend(positions)
    return all_positions

In [7]:
def intersections(positions1, positions2):
    return set(positions1) & set(positions2)

In [14]:
def nearest_intersection(intersections):
    return min(intersections, key=lambda p: manhattan_distance(p, (0, 0)))

Here are a few more examples:

```
R75,D30,R83,U83,L12,D49,R71,U7,L72
U62,R66,U55,R34,D71,R55,D58,R83 = distance 159
R98,U47,R26,D63,R33,U87,L62,D20,R33,U53,R51
U98,R91,D20,R16,D67,R40,U7,R15,U6,R7 = distance 135
```

In [9]:
def path(path_str):
    return [(m[0], int(m[1:])) for m in path_str.split(',')]

In [10]:
def test(wire1_path, wire2_path):
    path1 = path(wire1_path)
    path2 = path(wire2_path)
    positions1 = all_wire_positions(path1)
    positions2 = all_wire_positions(path2)
    all_intersections = intersections(positions1, positions2)
    nearest = nearest_intersection(all_intersections)
    return manhattan_distance(nearest, (0, 0))

In [15]:
assert test('R75,D30,R83,U83,L12,D49,R71,U7,L72', 'U62,R66,U55,R34,D71,R55,D58,R83') == 159
assert test('R98,U47,R26,D63,R33,U87,L62,D20,R33,U53,R51', 'U98,R91,D20,R16,D67,R40,U7,R15,U6,R7') == 135

What is the Manhattan distance from the central port to the closest intersection?

In [16]:
test(*data.strip().split('\n'))

1674

### Part 2
It turns out that this circuit is very timing-sensitive; you actually need to minimize the signal delay.

To do this, calculate the number of steps each wire takes to reach each intersection; choose the intersection where the sum of both wires' steps is lowest. If a wire visits a position on the grid multiple times, use the steps value from the first time it visits that position when calculating the total value of a specific intersection.

The number of steps a wire takes is the total number of grid squares the wire has entered to get to that location, including the intersection being considered. Again consider the example from above:

```
...........
.+-----+...
.|.....|...
.|..+--X-+.
.|..|..|.|.
.|.-X--+.|.
.|..|....|.
.|.......|.
.o-------+.
...........
```

In the above example, the intersection closest to the central port is reached after 8+5+5+2 = 20 steps by the first wire and 7+6+4+3 = 20 steps by the second wire for a total of 20+20 = 40 steps.

However, the top-right intersection is better: the first wire takes only 8+5+2 = 15 and the second wire takes only 7+6+2 = 15, a total of 15+15 = 30 steps.

In [26]:
def combined_path_to_intersection(positions1, positions2, point):
    # The puzzle counts the origin, which we havn't added up to this point, so add 2 (1 for each)
    return positions1.index(point) + positions2.index(point) + 2

In [20]:
def shortest_combined_path_intersection(positions1, positions2, intersections):
    return min(intersections, key=lambda p: combined_path_to_intersection(positions1, positions2, p))

In [21]:
def test_shortest_combined_path(wire1_path, wire2_path):
    path1 = path(wire1_path)
    path2 = path(wire2_path)
    positions1 = all_wire_positions(path1)
    positions2 = all_wire_positions(path2)
    all_intersections = intersections(positions1, positions2)
    nearest = shortest_combined_path_intersection(positions1, positions2, all_intersections)
    return combined_path_to_intersection(positions1, positions2, nearest)

Here are the best steps for the extra examples from above:

```
R75,D30,R83,U83,L12,D49,R71,U7,L72
U62,R66,U55,R34,D71,R55,D58,R83 = 610 steps
R98,U47,R26,D63,R33,U87,L62,D20,R33,U53,R51
U98,R91,D20,R16,D67,R40,U7,R15,U6,R7 = 410 steps
```

In [28]:
assert test_shortest_combined_path(
    'R75,D30,R83,U83,L12,D49,R71,U7,L72',
    'U62,R66,U55,R34,D71,R55,D58,R83'
) == 610
assert test_shortest_combined_path(
    'R98,U47,R26,D63,R33,U87,L62,D20,R33,U53,R51',
    'U98,R91,D20,R16,D67,R40,U7,R15,U6,R7'
) == 410

What is the fewest combined steps the wires must take to reach an intersection?

In [29]:
test_shortest_combined_path(*data.strip().split('\n'))

14012